# Session 5: Data munging with Pandas - Assignment

## [EAA - ARC Python Primer for Accounting Research](https://martien.netlify.app/book/example/)

### Using Pandas to explore, manage, clean data, deal with missing observations of US bank holding companies.
## Assignment
---

This assignment requires you to munge and explore accounting data of U.S. Bank Holding Companies. The reason I chose this data is that it offers freely available and highly structured accounting data, with a long history, comprising many banks.

The structure of the data is very similar to that of Compustat quarterly data. A couple of differences:

- entity variables are `RSSD9001` instead of `GVKEY`
- the data variable is `RSSD9999` instead of `datadate`
- Equity and Total Assets are `BHCK3210` and `BHCK2170` instead of `CEQ` and `AT`.
- there are thousands of variables, but many of them are missing.

The main source of data is the [Financial Data Download section](https://www.ffiec.gov/npw/FinancialReport/FinancialDataDownload?selectedyear=2020) of the [National Information Center](https://www.ffiec.gov/npw).

Individual Consolidated Financial Statements for Holding Companies on form *FR Y-9C*  can be downloaded via this [link](https://www.ffiec.gov/npw/).

For documentation  of the form *FR Y-9C*, see this [link](https://www.federalreserve.gov/apps/reportforms/reporthistory.aspx?sOoYJ+5BzDal8cbqnRxZRg==).

I encourage you to download an *FR Y-9C* copy, say, from Citigroup, via this [link](https://www.ffiec.gov/npw/Institution/Profile/1951350?dt=20170701). This enables you to quickly understand the variable names. 

The [Micro Data Reference Manual](https://www.federalreserve.gov/apps/mdrm/) gives you comprehensive dictionary of the variables used. 

The New York Fed maintains a file that links the BHC data to CRSP, see this [link](https://www.newyorkfed.org/research/banking_research/datasets.html).

**Required**:

From the FIC website, **download the BHC data** for [2019](https://www.ffiec.gov/npw/FinancialReport/FinancialDataDownload?selectedyear=2019),  [2020](https://www.ffiec.gov/npw/FinancialReport/FinancialDataDownload?selectedyear=2020), and  [2021](https://www.ffiec.gov/npw/FinancialReport/FinancialDataDownload?selectedyear=2021). 

**Save** the files to a folder on your drive, e.g. `D:/users/my_user_name_here/EAA_python/code/`. The files are zip- compressed `BHCF20200331.ZIP`, `BHCF20211231.ZIP` - **but there is no need to extract the contents of the zip files**. Pandas will do that for you.

Run the cells below after setting the correct source folder of your files, i.e. replace `my_user_name_here` with something that works on your machine. See this [link](https://www.youtube.com/watch?v=hUW5MEKDtMM) and this [link](https://www.youtube.com/watch?v=7ABkcHLdG_A) for explanations of folders and directories.

In [ ]:
# The usual preamble
import pandas as pd
import numpy as np
import os
# Some other modules
import glob

if os.name=='nt':  # for Windows users
    os.chdir('D:/users/my_user_name_here/EAA_python/data/')  # note the forward slashes, change 'my_user_name_here' to your user name
else:
    os.chdir('/home/my_user_name_here/EAA_python/data/')  # For Linux or Mac

In [ ]:
# Read the data file and append them into one large data frame:

def load_bhc_data():
    df = pd.DataFrame()
    for fname in glob.glob('BHCF*.ZIP'):
        print(fname)
        df = df.append(pd.read_csv(fname, sep='^', encoding="ISO-8859-1", low_memory=False))
    print(f'\nDone!\n\nTotal rows in data frame {len(df)}')
    print(f'\nTotal columns in data frame {len(list(df))}\nThat is a lot!')
    return df

df = load_bhc_data()        

---

**Required**: Use `.value_counts().sort_index()` to check how many observations per quarter your data has. Check this for the column `df['RSSD9999']` only.  

Follow this [link]((https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.value_counts.html)) for the correct syntax.

---

**Selecting variables**

---
Use your knowledge from [Session 3](https://github.com/blucap/EEA_Python_Primer/blob/master/session3.ipynb) and [Session 5](https://github.com/blucap/EEA_Python_Primer/blob/master/session5.ipynb) to keep the variables mentioned in the cell below.

**Hint**: use the text from cell below to make a dictionary (name that dictionary `mdrm`) and extract the keys from that dictionary (see [Session 3](https://github.com/blucap/EEA_Python_Primer/blob/master/session3.ipynb)) to make a list called  `var_list`. Use that list to create a new data frame with the relevant variables:  `dfb`: `df = df[var_list].copy()`

` 'RSSD9999': 'REPORTING DATE',
 'RSSD9001': 'Borrower RSSD ID',
 'RSSD9010': 'Entity Short Name',
 'RSSD9200': 'Abbreviated State Name',
 'RSSD4087': "Entity's World Wide Web Address",
 'BHCAP793': 'Common Equity Tier 1 Capital Ratio',
 'BHCKB529': 'Loans and Leases, Net of Unearned Income and Allowance',
 'BHCKB528': 'Loans and Leases, Net of Unearned Income - Totals',
 'BHCAA223': 'Risk-Weighted Assets (Net of Allowances and Other Deductions)',
 'BHCA8274': 'Tier 1 Capital Allowable Under the Risk-Based Capital Guidelines',
 'BHCA7206': 'Tier 1 Risk-Based Capital Ratio',
 'BHCA7204': 'Tier 1 Leverage Capital Ratio',
 'BHCA7205': 'Total Risk-Based Capital Ratio',
 'BHCK5369': 'Loans and Leases Held for Sale',
 'BHCK4635': 'Charge-Offs on Allowance for Loan and Lease Losses',
 'BHCK4605': 'Recoveries on Allowance for Loan and Lease Losses',
 'BHCK3210': 'Total Equity Capital',
 'BHCK3123': 'Allowance for Loan and Lease Losses',
 'BHCK2170': 'Total Assets',
 'BHCK1773': 'Available-for-Sale Debt Securities (From Schedule RC-B, Column D)',
 'BHCK0397': 'Interest-Bearing Balances in Foreign Offices, Edge and Agreement Subsidiaries and Ibfs',
 'BHCK0395': 'Interest-Bearing Balances in U.S. Offices',
 'BHCK0081': 'Noninterest-Bearing Balances and Currency and Coin'
`

---

**Eliminating rows that we don't need**

Using `dropna`, eliminate rows from the data frame with missing values for `BHCK3210` (equity) `BHCK2170` (total assets).

How many (or few) rows does the data frame have now?

In [ ]:
len(df)

---

**Setting a datadate column**

---

Use `pd.to_datetime(df['RSSD9999'])` to create a column named `datadate`. Use the format parameter to acknowledge the date format with year first, and day last (20190331):

Use `agg` to check the date range of ` datadate`: do the minimum and maximum values make sense? 

Use `df.datadate.value_counts()` to check the number of firms per quarter. 

In [ ]:
df.datadate.value_counts()

The last quarter (`'2021-12-31'`) has few useful rows. 

Using `.loc`, eliminate rows from that quarter:

In [ ]:
df = df.loc[df.datadate != ... ]
df.datadate.value_counts()

Note, sometimes it is easier to use `df.datadate` instead of `df['datadate']`. But it is generally better to stick to the latter format.

---

**Setting the index**

---

Set the index to `['RSSD9001', 'datadate']`: the bank identifier and the reporting date variable:

In [ ]:
# Confirm that it worked, the index should be in bold font
df.tail()


---
**Selecting data from individual banks.**

---

Using the index to find data from CitiGroup, which has id: *1951350*, works even if you have a multi-index

In [ ]:
citi = df.loc[1951350]
citi.head(2)

How would you select data from CitiGroup (*1951350*) and Bank of America (*1073757*)?

In [ ]:
citi_boa = ...
citi_boa.head()

In [ ]:
citi_boa.tail()

---

**Replacing values**

---

Note that `RSSD4087` (*Entity's World Wide Web Address*) is `'0'` in some rows. 

Mark these as missing values using the `replace` command, where you use Numpy's NaN (`NaN`) as the replacement value for '0':

In [ ]:
df['RSSD4087'].replace( ... , ..., inplace= True)

In [ ]:
df['RSSD4087']

**Required**: Now do the same for all rows with zero values for *risk-weighted assets* `BHCAA223`. 

**Hint**: *risk-weighted assets* are floats, i.e. numbers, not text; whereas the WWW address was string `'0'`.

In [ ]:
df['BHCAA223'].replace(... , ..., inplace= True)

Check that it worked, the minimum value should be > zero.

In [ ]:
df['BHCAA223'].min()

---

**How did bank liquidity change over recent quarters?**

---

In this part of the assignment you will calculate a liquidity ratio to measure trends in U.S. bank liquidity.

Define `cash` as the sum of these items: `BHCK0081` , `BHCK0395`, `BHCK0397`, and `BHCK1773`. 

You can check the item names using the dictionary `mdrm`. For example: 
`mdrm['BHCK0081']` for '*Noninterest-Bearing Balances and Currency and Coin*'.
    

In [ ]:
df['cash'] =  df[[... ... ...]].sum(axis='columns')

**Question**: What if we did not specify `axis='columns'` in the previous command?

Answer:

**Required**: Define `df['liq_ratio']` as the ratio of `cash` over total assets (`BHCK2170`).

In [ ]:
df['liq_ratio'] =  ...

**Question**: Is the data well-behaved, e.g. no outliers? 

Let's show the distribution of this variable using `hist()`. (See this [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html) for more on this command.)

In [ ]:
df['liq_ratio'].hist()

To plot the trend in bank liquidity over recent quarters, we can 

- use the approach shown in [Session 5](https://martien.netlify.app/slides/session5/), using the `resample` command, where we 'resample' the data per quarter and then take the mean of `liq_ratio`. However, we are not really resampling, because the frequency of the data is quarterly already. Moreover, if we resample we need to reset the index and set it to `datadate` only: `df = df.reset_index().set_index('datadate')`.
- Therefore, **it is better to group the data by `datadate`**, using `groupby`, then take the mean, and plot. In that case we do not need to reset the index.

In [ ]:
# Using resample - will thow an error, so I commented it out.
# df['liq_ratio'].resample('Q').mean().plot()

In [ ]:
# Using groupby
df['liq_ratio'].groupby(...). ...().plot()

**Required**: Make the figure larger using `figsize=(8,6)` and add a meaningful title.

See this [link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) for documentation on `plot`.

In [ ]:
df['liq_ratio'].groupby('datadate').mean().plot(... , ...)

We can store the underlying data in a separate frame:

In [ ]:
df_lr = df['liq_ratio'].groupby('datadate').mean()
df_lr

---

**How did bank capital ratios change over recent quarters?**

---

To answer this question use, the same approach as above, but instead of defining variables we can rely on these two items: 
- `BHCA7206`: 'Tier 1 Risk-Based Capital Ratio'
- `BHCA7204`: 'Tier 1 Leverage Capital Ratio'
- `BHCA7205`: 'Total Risk-Based Capital Ratio'.

In [ ]:
df[... ... ...].groupby(...).mean().plot()

**Question**: In early 2020, the Tier 1 ratio dropped faster than the leverage ratio. Why?

---

**Question**: Can we improve the presentation of the graph by using the variable labels instead of the mnemonics?

**Hint**: one way is use the `mdrm` dictionary to rename the columns `['BHCA7206', 'BHCA7204', 'BHCA7205']`. See this [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) for documentation on `rename`.

In [ ]:
dfl = df[... ... ...].groupby(...).mean()
dfl.rename(columns = ..., inplace = True)
dfl.plot()

---


**Joining data and counting the number of BHCs per state**

---

We can use the column `RSSD9200` (*Abbreviated State Name*) to count the number of BHCs per state. I want you to use state names instead. There is a GitHub site that can help us with this: click [here](https://gist.github.com/JeffPaine/3083347). The site offers a dictionary, which we can convert into a data frame, and then join with our main data frame.

Copy that dictionary and complete the function that converts it to a dictionary that we can merge:

In [ ]:
def state_names():
    states = {'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'}
    states = pd.DataFrame.from_dict(states, orient ='index', columns=['State name'])
    # The index of this data frame has no name, so we should name the index:
    states.index.name='RSSD9200'
    return states

states = state_names()
states.head(3)

We can now join straight away, without needing to set a key for the left-hand data frame (!)

The only thing we need to do is to specify the relevant key column name from the left-hand data frame: '`RSSD9200`'. The advantage of this approach is that we do not need to set and reset the index.

In [ ]:
df = df.join(states, on='RSSD9200')

In [ ]:
df.head(2)

---

**Question**: What is wrong with the command below if we want to know the number of banks in our sample for a given year?

In [ ]:
df.groupby('State name')['RSSD9010'].count().head(5)

**Required**: We can do better, by selecting a subset of the data: `df.loc[df['RSSD9999']==20210630, :]` and then apply the `groupby` statement from the previous cell.

In [ ]:
dfc = ... .groupby('State name')['RSSD9010'].count()
dfc

**Bonus question**: How to turn the data from `dfc`  into a bar-plot with figure size (10 x 6) and title 'BHCs'?

Use `.sort_values(ascending = False)` to sort `dfc`.

See this [link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) for documentation on `plot`.